# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [9]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [ ]:
from sklearn.model_selection import train_test_split
# Create X and Y
X = adult_dt.drop(columns = ['income'])
Y = adult_dt['income']
# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*
The random_state parameter in the train_test_split function ensures that the data is split in a reproducible manner.   
This is useful because it allows us to compare the performance of different models on the same data split.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define numerical and categorical lists
num_var = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


# Define the column transformer
ct = ColumnTransformer(
    transformers = [
        ('num_transforms', 
         Pipeline(steps = [
             ('imputer', KNNImputer(n_neighbors = 7, weights = 'distance')),
             ('scaler', RobustScaler())
         ]), 
         num_var
        ),
        ('cat_transforms', 
         Pipeline(steps = [
             ('imputer', SimpleImputer(strategy = 'most_frequent')),
             ('ohe', OneHotEncoder(handle_unknown = 'ignore', drop = 'if_binary'))
         ]), 
         cat_var
        )
    ]
)
ct

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [32]:
from sklearn.ensemble import RandomForestClassifier

# Define the pipeline
model_pipeline = Pipeline (steps = [
    ('preprocessing', ct),  # Adding the preprocessing step with the ColumnTransformer
    ('classifier', RandomForestClassifier()) 
])
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [ ]:
from sklearn.model_selection import cross_validate

# Define the metrics
scoring_metrics = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Perform 5 cross-validation
cross_validation = cross_validate (model_pipeline, X_train, Y_train, cv = 5, scoring = scoring_metrics, return_train_score=True)

# Print results
cross_validation

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([4.24662995, 4.26847124, 4.24531198, 4.20857501, 4.21132278]),
 'score_time': array([0.15009499, 0.15152693, 0.14972115, 0.14948082, 0.14832306]),
 'test_neg_log_loss': array([-0.34979342, -0.3527517 , -0.40332038, -0.36593739, -0.40196909]),
 'train_neg_log_loss': array([-0.08144173, -0.08122494, -0.08129286, -0.0820959 , -0.08116054]),
 'test_roc_auc': array([0.90467929, 0.90337765, 0.90132444, 0.90585012, 0.9015273 ]),
 'train_roc_auc': array([1., 1., 1., 1., 1.]),
 'test_accuracy': array([0.85303795, 0.84821233, 0.85234752, 0.86046512, 0.85585783]),
 'train_accuracy': array([1.        , 1.        , 0.99994516, 1.        , 1.        ]),
 'test_balanced_accuracy': array([0.77884262, 0.76828232, 0.77393531, 0.78452366, 0.77471371]),
 'train_balanced_accuracy': array([1.        , 1.        , 0.99988693, 1.        , 1.        ])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [46]:
# Convert the results to a DataFrame
results_df = pd.DataFrame(cross_validation)

# Sort by test negative log loss (ascending)
results_df.sort_values('test_neg_log_loss' , ascending = True)

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
2,4.245312,0.149721,-0.403320,-0.081293,0.901324,1.0,0.852348,0.999945,0.773935,0.999887
4,4.211323,0.148323,-0.401969,-0.081161,0.901527,1.0,0.855858,1.000000,0.774714,1.000000
3,4.208575,0.149481,-0.365937,-0.082096,0.905850,1.0,0.860465,1.000000,0.784524,1.000000
1,4.268471,0.151527,-0.352752,-0.081225,0.903378,1.0,0.848212,1.000000,0.768282,1.000000
0,4.246630,0.150095,-0.349793,-0.081442,0.904679,1.0,0.853038,1.000000,0.778843,1.000000


Calculate the mean of each metric. 

In [ ]:
# Calculate mean metrics
mean_metric = results_df.mean()
mean_metric


fit_time                   4.236062
score_time                 0.149829
test_neg_log_loss         -0.374754
train_neg_log_loss        -0.081443
test_roc_auc               0.903352
train_roc_auc              1.000000
test_accuracy              0.853984
train_accuracy             0.999989
test_balanced_accuracy     0.776060
train_balanced_accuracy    0.999977
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [54]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the pipeline model
model_pipeline.fit(X_train, Y_train)
# Make predictions
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the metrics
results_test = {
    'Accuracy': accuracy_score(Y_test, Y_pred),
    'Balanced Accuracy': balanced_accuracy_score(Y_test, Y_pred),
    'ROC AUC': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'Negative Log Loss': -log_loss(Y_test, Y_pred_proba)
}

# Display the results
results_test


{'Accuracy': 0.8562800696079435,
 'Balanced Accuracy': 0.7760499325541262,
 'ROC AUC': 0.8995368059697858,
 'Negative Log Loss': -0.38467050084117943}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)   
It makes the target variable usable for classification algorithms and allows for the calculation of proper performance metrics.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.